In [2]:
import pandas as pd
import numpy as np
import import_ipynb
import sys
import math 
import seaborn as sea
import datetime as dt
import Fuzzy_Prediction_Beta as fz
from datetime import timedelta
from operator import itemgetter
from functools import partial
from statistics import mean 

url = "in.csv"

dataset = pd.read_csv(url, error_bad_lines=False)

selected = dataset[['cod_location','date_id','total_deaths']]

rasuto = selected.groupby('cod_location').tail(5)
selected = selected.groupby('cod_location').tail(10)
selected = selected.groupby('cod_location').head(5)

selected.head()
#Input na rede neural
selected.to_csv('input_d.csv')

def aver(input):
    medias = input.groupby('cod_location')['total_deaths'].apply(lambda x: x.tail(5).mean())
    return medias

def diff(input):
    difference = input.groupby('cod_location')['total_deaths'].apply(lambda x: x.tail(1).values[0] - x.head(1).values[0])
    return difference

def first(input):
    first = selected.groupby('cod_location')['total_deaths'].apply(lambda x: x.head(1).values[0])
    return first

def last(input):
    last = selected.groupby('cod_location')['total_deaths'].apply(lambda x: x.tail(1).values[0])
    return last

def firsta(input):
    first = rasuto.groupby('cod_location')['total_deaths'].apply(lambda x: x.head(1).values[0])
    return first

def lasto(input):
    last = rasuto.groupby('cod_location')['total_deaths'].apply(lambda x: x.tail(1).values[0])
    return last

def param(last,diffe):
    param = last + diffe
    return param

media = aver(selected)
diffe = diff(selected)
first = first(selected)
last = last(selected)
param =  last + diffe
maximus = max(selected['total_deaths'])

mediaa = aver(rasuto)
diffee = diff(rasuto)
firsta = firsta(rasuto)
lastt = lasto(rasuto)
paramm =  lastt + diffee
maximuss = max(rasuto['total_deaths'])

centroid_fore_data = pd.DataFrame()
centroid_act_data = pd.DataFrame()

for i in range(213):
    Regras = fz.verifRegras(param[i], media[i])
    centroid_fore = fz.getCentroid(media[i],Regras,param[i])
    centroid_fore_data = centroid_fore_data.append({'cd_local':i,'centroid':centroid_fore}, ignore_index=True)
    Rules = fz.verifRegras(paramm[i], mediaa[i])
    centroid_act = fz.getCentroid(mediaa[i],Rules,paramm[i])
    centroid_act_data = centroid_act_data.append({'cd_local':i,'centroid':centroid_act}, ignore_index=True)

Error = ((abs(centroid_act_data['centroid'] - centroid_fore_data['centroid']) / centroid_act_data['centroid']) * 100)

where_are_NaN = np.isnan(Error)
Error[where_are_NaN] = 0

where_are_in = np.isinf(Error)
Error[where_are_in] = 100


for i in range(213):
    if Error[i] > 100:
        Error[i] = 100

Error.mean()

19.591474957043836